
**Required Libraries**

In [1]:
import xml.etree.ElementTree as ET
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d
import os
from numpy.linalg import norm
import ipywidgets as widgets
from ipywidgets import interact


**The dGLI Calculation Function**

In [2]:
def selectedEdges(edges, edge_indices):
    """Selects edges from the given list based on the provided indices."""
    outCurve = []
    for index in edge_indices:
        outCurve.append(edges[index])
    return outCurve

def dGLICoords(edgesCurve1):
    return dGLICoords2(edgesCurve1, edgesCurve1)

def dGLICoords2(edgesCurve1, edgesCurve2):
    numP1 = len(edgesCurve1)
    numP2 = len(edgesCurve2)
    vector_res = []
    for i in range(numP1):
        for j in range(i, numP2):
            if i!=j:
                dgli = dGLI(edgesCurve1[i][0], edgesCurve1[i][1], edgesCurve2[j][0], edgesCurve2[j][1])
                vector_res.append(dgli)
    return vector_res

def dGLI(ap1, ap2, ap3, ap4):
    epsilon = 0.00001
    p1 = np.add(ap1, [0, 0, 0])
    p2 = np.add(ap2, [0, 0, epsilon])
    p3 = np.add(ap3, [0, 0, 0])
    p4 = np.add(ap4, [0, 0, epsilon])
    if np.linalg.norm(np.subtract(ap1, ap3)) < 0.01 and np.linalg.norm(np.subtract(ap2, ap4)) < 0.01:
        ret = 0
    else:
        GLI = evalWritheTwoSegments(ap1, ap2, ap3, ap4)
        GLIPerturbed = evalWritheTwoSegments(p1, p2, p3, p4)
        ret = (GLIPerturbed - GLI) / epsilon
    if abs(ret) > 4.0:
        return np.sign(ret) * 4
    else:
        return ret

def evalWritheTwoSegments(p1, p2, p3, p4):
    n1 = np.cross(np.subtract(p3, p1), np.subtract(p4, p1)) / np.linalg.norm(np.cross(np.subtract(p3, p1), np.subtract(p4, p1)))
    n2 = np.cross(np.subtract(p4, p1), np.subtract(p4, p2)) / np.linalg.norm(np.cross(np.subtract(p4, p1), np.subtract(p4, p2)))
    n3 = np.cross(np.subtract(p4, p2), np.subtract(p3, p2)) / np.linalg.norm(np.cross(np.subtract(p4, p2), np.subtract(p3, p2)))
    n4 = np.cross(np.subtract(p3, p2), np.subtract(p3, p1)) / np.linalg.norm(np.cross(np.subtract(p3, p2), np.subtract(p3, p1)))
    sgn = np.sign(np.dot(np.cross(np.subtract(p4, p3), np.subtract(p2, p1)), np.subtract(p3, p1)))
    ret = np.arcsin(np.dot(n1, n2)) + np.arcsin(np.dot(n2, n3)) + np.arcsin(np.dot(n3, n4)) + np.arcsin(np.dot(n4, n1))
    return sgn * ret




**Upload the dataset with the bag mesh coordinates.**

In [3]:
df=pd.read_csv("bag_mesh_coordinates_dataset.csv")
df1=df.to_numpy()[:,0:4]

In [4]:
def frame(frame):
    df2=df1[df1[:,0]==frame]
    df_frame=df2[:,1:4]
    return df_frame

In [5]:
def plot_frame2(c):
    fig = plt.figure()
    ax = plt.axes(projection='3d')
    ax.scatter3D(*zip(*frame(c)))
    ax.axis('equal')
    ax.view_init(elev=30, azim=45)
    ax.set_title("Cloth mesh plot")
    ax.set_xlabel('x-axis')
    ax.set_ylabel('y-axis')
    ax.set_zlabel('z-axis')
    ax.set_xlim(-2.2,2.2)
    ax.set_ylim(-1.4,2.4)
    ax.set_zlim(-1,1.6)
    plt.show()

**Upload the dataset with the selected bag mesh coordinates.**

In [7]:
sel_df=pd.read_csv("bag_mesh_selected_coordinates_dataset.csv")
sel_df1=sel_df.to_numpy()[:,0:4]

In [8]:
def sel_frame(sel_frame):
    sel_df2=sel_df1[sel_df1[:,0]==sel_frame]
    sel_df_frame=sel_df2[:,1:4]
    return sel_df_frame

In [9]:
def distance(p1, p2):
    return np.linalg.norm(p1 - p2)

def order_coordinates(coords):
    coords = [list(c) for c in coords]
    ordered = [coords.pop(0)]

    while coords:
        last_point = ordered[-1]
        closest_point = min(coords, key=lambda point: distance(np.array(last_point), np.array(point)))
        ordered.append(closest_point)
        coords.remove(closest_point)

    return np.array(ordered)

def frames(p):
    list_of_lists = order_coordinates(sel_frame(p)).tolist()
    nested_list = [list_of_lists[i:i+2] for i in range(0, len(list_of_lists), 2)]
    return nested_list


**Visualization**

In [10]:
from matplotlib.colors import LinearSegmentedColormap
def plot_frame1(l):
    matrix = np.full((8, 8), np.nan)
    k = 0
    for i in range(8):
        for j in range(i+1, 8):
            matrix[i, j] = dGLICoords(frames(l))[k]
            k += 1

    vmin = -0.5
    vmax = 0.5

    colors = [(0, 0, 1), (1, 0.5, 0)]
    n_bins = [3, 6, 10, 100]
    cmap_name = 'custom_div_cmap'
    cm = LinearSegmentedColormap.from_list(cmap_name, colors, N=100)

    plt.imshow(matrix, cmap=cm, interpolation='nearest', origin='upper', vmin=vmin, vmax=vmax)
    plt.colorbar()
    plt.tick_params(left=False, right=True, bottom=False, top=True, labelleft=False, labelbottom=False, labeltop=True, labelright=True)
    labels = [str(i+1) for i in range(8)]
    plt.xticks(range(8), labels)
    plt.yticks(range(8), labels)
    plt.title('Upper Triangular Matrix')
    plt.grid(False)
    plt.show()

def plot_frame(c):
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')

    for idx, edge in enumerate(frames(c)):
        x = [point[0] for point in edge]
        y = [point[1] for point in edge]
        z = [point[2] for point in edge]
        ax.plot(x, y, z)

        mid_x = sum(x) / 2
        mid_y = sum(y) / 2
        mid_z = sum(z) / 2

        ax.text(mid_x, mid_y, mid_z, str(idx+1), color='red', fontsize=10, weight='bold')

    ax.view_init(elev=30, azim=45)
    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    ax.set_zlabel('Z')
    ax.set_xlim(-2.2,2.2)
    ax.set_ylim(-1.4,2.4)
    ax.set_zlim(-1,1.6)
    plt.show()

In [11]:
c_slider = widgets.IntSlider(value=0, min=1, max=350, step=1, description='c value:')
interact(plot_frame1, l=c_slider);
c_slider1 = widgets.IntSlider(value=0, min=1, max=350, step=1, description='c1 value:')
interact(plot_frame, c=c_slider1);
c_slider2 = widgets.IntSlider(value=0, min=0, max=350, step=16, description='c1 value:')
interact(plot_frame2, c=c_slider2);

interactive(children=(IntSlider(value=1, description='c value:', max=350, min=1), Output()), _dom_classes=('wi…

interactive(children=(IntSlider(value=1, description='c1 value:', max=350, min=1), Output()), _dom_classes=('w…

interactive(children=(IntSlider(value=0, description='c1 value:', max=350, step=16), Output()), _dom_classes=(…